In [1]:
import pandas as pd
import os
import numpy as np
import matplotlib.pyplot as plt

In [2]:
train = pd.read_csv("../../../card/train.csv",index_col=0)
test = pd.read_csv("../../../card/test.csv",index_col=0)
sub  = pd.read_csv('../../../card/sample_submission.csv')

In [3]:
train.loc[train.DAYS_EMPLOYED >0,'DAYS_EMPLOYED'] =0
test.loc[test.DAYS_EMPLOYED >0,'DAYS_EMPLOYED'] =0

In [4]:
NumericCol=train.select_dtypes(include=np.number).columns
StrCol=train.select_dtypes(include=object).columns

In [5]:
from sklearn.ensemble import RandomForestClassifier

In [6]:
def checkUniqueObject(train,test):
    target_col = train.select_dtypes(include= object).columns
    
    check =0
    for col in target_col:
        trainU = train[col].unique()
        testU  = test[col].unique()

        if set(trainU) != set(testU):
            print(col,' have not same Unique values')
            check=1
            
    if check ==0:
        print('all columns have Same Unique values')

checkUniqueObject(train,test)

all columns have Same Unique values


In [7]:
def LabelEn(train,test):
    trainLa = train.copy()
    testLa  = test.copy()
    
    from sklearn.preprocessing import LabelEncoder
    target_col = train.select_dtypes(include= object).columns
    for col in target_col:
        le = LabelEncoder()
        le.fit(trainLa[col])
        trainLa[col] = le.transform(trainLa[col])
        testLa[col] = le.transform(testLa[col])
        
        
    
    return trainLa , testLa

In [8]:
trainLa , testLa = LabelEn(train,test)

In [9]:
x = trainLa.drop('credit',axis=1)
y = trainLa['credit']

In [10]:
from sklearn.model_selection import train_test_split

In [41]:
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.1, random_state=42)

In [12]:
rf = RandomForestClassifier(random_state=43)

In [13]:
rf.fit(X_train,y_train)

RandomForestClassifier(random_state=43)

In [14]:
from sklearn.metrics import log_loss

In [15]:
clf_probs = rf.predict_proba(X_test)
score = log_loss(y_test, clf_probs)

score

0.9974201226349136

In [16]:
final_Pred = rf.predict_proba(testLa)

sub  = pd.read_csv('../../../card/sample_submission.csv')
sub[['0','1','2']] =final_Pred
sub.to_csv("submission.csv",index=0)

In [44]:
import lightgbm as lgb
from scipy.stats import randint as sp_randint
from scipy.stats import uniform as sp_uniform
import lightgbm as lgb
from sklearn.model_selection import RandomizedSearchCV, GridSearchCV


In [96]:

def model_kaggle(train_set,label,feature_name_lst,seed1):
    name = str(datetime.datetime.now())[:-10] +""
    model_name = 'one_que'

    random_seed_change =314
    from sklearn.model_selection import train_test_split
    X_train, X_test, y_train, y_test = train_test_split(train_set, label ,test_size=0.20, random_state=random_seed_change, stratify=label)



    def learning_rate_010_decay_power_099(current_iter):
        base_learning_rate = 0.1
        lr = base_learning_rate  * np.power(.99, current_iter)
        return lr if lr > 1e-4 else 1e-4

    def learning_rate_010_decay_power_0995(current_iter):
        base_learning_rate = 0.1
        lr = base_learning_rate  * np.power(.995, current_iter)
        return lr if lr > 1e-4 else 1e-4

    def learning_rate_005_decay_power_099(current_iter):
        base_learning_rate = 0.05
        lr = base_learning_rate  * np.power(.99, current_iter)
        return lr if lr > 1e-3 else 1e-3

    def learning_rate_010_decay_power_0996(current_iter):
        base_learning_rate = 0.1
        lr = base_learning_rate  * np.power(.996, current_iter)
        return lr if lr > 1e-4 else 1e-4

    
    fit_params={"early_stopping_rounds":5, 
                "eval_metric" : 'multi_logloss', 
                "eval_set" : [(X_test,y_test)],
                'eval_names': ['valid'],
                'callbacks': [lgb.reset_parameter(learning_rate=learning_rate_010_decay_power_0996)],
                'verbose': 100,
                'categorical_feature': 'auto'}




    param_test ={'num_leaves': sp_randint(6, 50), 
                 'min_child_samples': sp_randint(100, 500), 
                 'min_child_weight': [1e-5, 1e-3, 1e-2, 1e-1, 1, 1e1, 1e2, 1e3, 1e4],
                 'subsample': sp_uniform(loc=0.2, scale=0.8), 
                 'colsample_bytree': sp_uniform(loc=0.4, scale=0.6),
                 'reg_alpha': [0, 1e-1, 1, 2, 5, 7, 10, 50, 100],
                 'reg_lambda': [0, 1e-1, 1, 5, 10, 20, 50, 100]}


    n_HP_points_to_test = 100


    #n_estimators is set to a "large value". The actual number of trees build will depend on early stopping and 5000 define only the absolute maximum
    clf = lgb.LGBMClassifier(max_depth=-1, random_state=seed1, silent=True, metric='None', n_jobs=4, n_estimators=1000)
    gs = RandomizedSearchCV(
        estimator=clf, param_distributions=param_test, 
        n_iter=n_HP_points_to_test,
        scoring='roc_auc',
        cv=3,
        refit=True,
        random_state=random_seed_change,
        verbose=True)



    gs.fit(X_train, y_train, **fit_params)
    print('Best score reached: {} with params: {} '.format(gs.best_score_, gs.best_params_))

    opt_parameters = gs.best_params_
    clf_final = lgb.LGBMClassifier(**clf.get_params())
    clf_final.set_params(**opt_parameters)

    #Train the final model with learning rate decay
    fit_params_final={"early_stopping_rounds":100, 
                "eval_metric" : 'multi_logloss', 
                "eval_set" : [(X_test,y_test)],
                'eval_names': ['valid'],
                'callbacks': [lgb.reset_parameter(learning_rate=learning_rate_010_decay_power_0995)],
                'verbose': 100,
                'categorical_feature': 'auto'}
    
    
    
    clf_final.fit(X_train, y_train, **fit_params_final) #,feature_name =feature_name_lst)

    return clf_final


In [97]:
train_x  =x
train_y =y
label = train_y

In [98]:
w=''

models33 =model_kaggle(train_x,label,314,w)

Fitting 3 folds for each of 100 candidates, totalling 300 fits
Training until validation scores don't improve for 5 rounds
[100]	valid's multi_logloss: 0.79534
[200]	valid's multi_logloss: 0.789462
[300]	valid's multi_logloss: 0.785725
Early stopping, best iteration is:
[344]	valid's multi_logloss: 0.785043
Training until validation scores don't improve for 5 rounds
[100]	valid's multi_logloss: 0.79326
[200]	valid's multi_logloss: 0.785937
[300]	valid's multi_logloss: 0.783216
[400]	valid's multi_logloss: 0.781222
[500]	valid's multi_logloss: 0.780072
[600]	valid's multi_logloss: 0.779161
Early stopping, best iteration is:
[615]	valid's multi_logloss: 0.779088
Training until validation scores don't improve for 5 rounds
[100]	valid's multi_logloss: 0.793701
[200]	valid's multi_logloss: 0.787788
[300]	valid's multi_logloss: 0.78497
[400]	valid's multi_logloss: 0.783135
[500]	valid's multi_logloss: 0.782015
[600]	valid's multi_logloss: 0.781277
[700]	valid's multi_logloss: 0.780707
Early 

[100]	valid's multi_logloss: 0.795448
[200]	valid's multi_logloss: 0.790344
[300]	valid's multi_logloss: 0.788419
Early stopping, best iteration is:
[335]	valid's multi_logloss: 0.787977
Training until validation scores don't improve for 5 rounds
[100]	valid's multi_logloss: 0.792274
[200]	valid's multi_logloss: 0.785874
Early stopping, best iteration is:
[207]	valid's multi_logloss: 0.785521
Training until validation scores don't improve for 5 rounds
[100]	valid's multi_logloss: 0.788602
[200]	valid's multi_logloss: 0.782085
[300]	valid's multi_logloss: 0.778487
[400]	valid's multi_logloss: 0.776646
Early stopping, best iteration is:
[397]	valid's multi_logloss: 0.776608
Training until validation scores don't improve for 5 rounds
[100]	valid's multi_logloss: 0.790263
[200]	valid's multi_logloss: 0.784437
[300]	valid's multi_logloss: 0.781884
[400]	valid's multi_logloss: 0.780339
Early stopping, best iteration is:
[418]	valid's multi_logloss: 0.780196
Training until validation scores d

Training until validation scores don't improve for 5 rounds
[100]	valid's multi_logloss: 0.805627
[200]	valid's multi_logloss: 0.800802
[300]	valid's multi_logloss: 0.79909
[400]	valid's multi_logloss: 0.798172
Early stopping, best iteration is:
[417]	valid's multi_logloss: 0.798018
Training until validation scores don't improve for 5 rounds
[100]	valid's multi_logloss: 0.80362
[200]	valid's multi_logloss: 0.798383
[300]	valid's multi_logloss: 0.79599
[400]	valid's multi_logloss: 0.794634
[500]	valid's multi_logloss: 0.793836
[600]	valid's multi_logloss: 0.793362
[700]	valid's multi_logloss: 0.793006
Early stopping, best iteration is:
[734]	valid's multi_logloss: 0.79291
Training until validation scores don't improve for 5 rounds
[100]	valid's multi_logloss: 0.802626
[200]	valid's multi_logloss: 0.798283
[300]	valid's multi_logloss: 0.796236
[400]	valid's multi_logloss: 0.795258
[500]	valid's multi_logloss: 0.794615
Early stopping, best iteration is:
[510]	valid's multi_logloss: 0.7945

[300]	valid's multi_logloss: 0.811972
Early stopping, best iteration is:
[343]	valid's multi_logloss: 0.811964
Training until validation scores don't improve for 5 rounds
[100]	valid's multi_logloss: 0.905886
[200]	valid's multi_logloss: 0.904877
Early stopping, best iteration is:
[216]	valid's multi_logloss: 0.904858
Training until validation scores don't improve for 5 rounds
[100]	valid's multi_logloss: 0.798409
[200]	valid's multi_logloss: 0.795733
Early stopping, best iteration is:
[205]	valid's multi_logloss: 0.795552
Training until validation scores don't improve for 5 rounds
[100]	valid's multi_logloss: 0.797563
[200]	valid's multi_logloss: 0.792264
Early stopping, best iteration is:
[224]	valid's multi_logloss: 0.791283
Training until validation scores don't improve for 5 rounds
[100]	valid's multi_logloss: 0.796946
[200]	valid's multi_logloss: 0.793729
Early stopping, best iteration is:
[233]	valid's multi_logloss: 0.79325
Training until validation scores don't improve for 5 r

[100]	valid's multi_logloss: 0.79876
Early stopping, best iteration is:
[139]	valid's multi_logloss: 0.796835
Training until validation scores don't improve for 5 rounds
[100]	valid's multi_logloss: 0.797726
[200]	valid's multi_logloss: 0.791835
[300]	valid's multi_logloss: 0.790132
Early stopping, best iteration is:
[309]	valid's multi_logloss: 0.789973
Training until validation scores don't improve for 5 rounds
[100]	valid's multi_logloss: 0.797154
[200]	valid's multi_logloss: 0.793249
Early stopping, best iteration is:
[195]	valid's multi_logloss: 0.793235
Training until validation scores don't improve for 5 rounds
Early stopping, best iteration is:
[1]	valid's multi_logloss: 1.03544
Training until validation scores don't improve for 5 rounds
Early stopping, best iteration is:
[1]	valid's multi_logloss: 1.03544
Training until validation scores don't improve for 5 rounds
Early stopping, best iteration is:
[2]	valid's multi_logloss: 1.03559
Training until validation scores don't impro

[100]	valid's multi_logloss: 0.801152
[200]	valid's multi_logloss: 0.795746
[300]	valid's multi_logloss: 0.793683
[400]	valid's multi_logloss: 0.792405
[500]	valid's multi_logloss: 0.791878
Early stopping, best iteration is:
[526]	valid's multi_logloss: 0.791775
Training until validation scores don't improve for 5 rounds
Early stopping, best iteration is:
[91]	valid's multi_logloss: 0.799461
Training until validation scores don't improve for 5 rounds
[100]	valid's multi_logloss: 0.799344
Early stopping, best iteration is:
[139]	valid's multi_logloss: 0.797494
Training until validation scores don't improve for 5 rounds
[100]	valid's multi_logloss: 0.799071
Early stopping, best iteration is:
[137]	valid's multi_logloss: 0.797559
Training until validation scores don't improve for 5 rounds
[100]	valid's multi_logloss: 0.804359
[200]	valid's multi_logloss: 0.799246
Early stopping, best iteration is:
[279]	valid's multi_logloss: 0.798015
Training until validation scores don't improve for 5 r

Training until validation scores don't improve for 5 rounds
Early stopping, best iteration is:
[2]	valid's multi_logloss: 1.03559
Training until validation scores don't improve for 5 rounds
Early stopping, best iteration is:
[85]	valid's multi_logloss: 0.858315
Training until validation scores don't improve for 5 rounds
Early stopping, best iteration is:
[33]	valid's multi_logloss: 0.857831
Training until validation scores don't improve for 5 rounds
Early stopping, best iteration is:
[30]	valid's multi_logloss: 0.858216
Training until validation scores don't improve for 5 rounds
[100]	valid's multi_logloss: 0.813067
[200]	valid's multi_logloss: 0.812144
[300]	valid's multi_logloss: 0.812036
[400]	valid's multi_logloss: 0.812003
[500]	valid's multi_logloss: 0.811989
[600]	valid's multi_logloss: 0.811983
[700]	valid's multi_logloss: 0.81198
[800]	valid's multi_logloss: 0.811978
[900]	valid's multi_logloss: 0.811977
[1000]	valid's multi_logloss: 0.811977
Did not meet early stopping. Best 

[500]	valid's multi_logloss: 0.811984
[600]	valid's multi_logloss: 0.811983
[700]	valid's multi_logloss: 0.811983
[800]	valid's multi_logloss: 0.811983
[900]	valid's multi_logloss: 0.811982
[1000]	valid's multi_logloss: 0.811982
Did not meet early stopping. Best iteration is:
[1000]	valid's multi_logloss: 0.811982
Training until validation scores don't improve for 5 rounds
[100]	valid's multi_logloss: 0.811732
[200]	valid's multi_logloss: 0.811559
[300]	valid's multi_logloss: 0.81155
[400]	valid's multi_logloss: 0.811548
[500]	valid's multi_logloss: 0.811548
[600]	valid's multi_logloss: 0.811547
[700]	valid's multi_logloss: 0.811547
[800]	valid's multi_logloss: 0.811547
[900]	valid's multi_logloss: 0.811547
[1000]	valid's multi_logloss: 0.811547
Did not meet early stopping. Best iteration is:
[1000]	valid's multi_logloss: 0.811547
Training until validation scores don't improve for 5 rounds
[100]	valid's multi_logloss: 0.800385
Early stopping, best iteration is:
[130]	valid's multi_logl

Early stopping, best iteration is:
[271]	valid's multi_logloss: 0.791011
Training until validation scores don't improve for 5 rounds
[100]	valid's multi_logloss: 0.797442
[200]	valid's multi_logloss: 0.793176
Early stopping, best iteration is:
[243]	valid's multi_logloss: 0.792189
Training until validation scores don't improve for 5 rounds
[100]	valid's multi_logloss: 0.792764
[200]	valid's multi_logloss: 0.787203
[300]	valid's multi_logloss: 0.785011
Early stopping, best iteration is:
[304]	valid's multi_logloss: 0.784953
Training until validation scores don't improve for 5 rounds
[100]	valid's multi_logloss: 0.792863
[200]	valid's multi_logloss: 0.787351
[300]	valid's multi_logloss: 0.784216
[400]	valid's multi_logloss: 0.782415
[500]	valid's multi_logloss: 0.781126
Early stopping, best iteration is:
[533]	valid's multi_logloss: 0.780798
Training until validation scores don't improve for 5 rounds
[100]	valid's multi_logloss: 0.791233
[200]	valid's multi_logloss: 0.785279
[300]	valid'

In [101]:
final_Pred = models33.predict_proba(testLa)

sub  = pd.read_csv('../../../card/sample_submission.csv')
sub[['0','1','2']] =final_Pred
sub.to_csv("submission.csv",index=0)